In [3]:
import os 
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import json
llc_accesses = pd.read_csv('caches/llc_accesses.csv')
llc_misses = pd.read_csv('caches/llc_misses.csv')
min_size = 512 * 1024
out_df = pd.read_csv('output_512/dataset.csv')
# print(out_df)
for file in os.listdir('reuse_distances'):
    if (file[:-4]+"_0")in set(out_df["id"]):
        print(file)
        print(123)
        continue
    rd_df = pd.read_csv('reuse_distances/'+file)
    ctr = 0
    miss_ctr = 0
    prev_ctr = 0
    for i in range(len(rd_df)):
        my_id = rd_df.iloc[i]
        ctr += int(llc_accesses[llc_accesses["id"] == my_id["id"]]["1024"])
        miss_ctr += int(llc_misses[llc_misses["id"] == my_id["id"]]["1024"])
        if ctr > min_size:
            out_line = rd_df.iloc[prev_ctr: i+1].sum(axis=0)
            out_line["id"] = file[:-4] + '_' + str(prev_ctr)
            out_line = pd.DataFrame(out_line).transpose()
            out_line["accesses_1024"] = ctr
            out_line["misses_1024"] = miss_ctr
            out_df = pd.concat([out_df, out_line], ignore_index = True)
            prev_ctr = i+1
            ctr = 0
            miss_ctr = 0
    print(out_df)
    out_df.to_csv('output_512/dataset.csv',index=False)

400.perlbench-41B.csv
123
400.perlbench-50B.csv
123
401.bzip2-226B.csv
123
401.bzip2-277B.csv
123
401.bzip2-38B.csv
123
401.bzip2-7B.csv
123
403.gcc-16B.csv
123
403.gcc-17B.csv
123
403.gcc-48B.csv
123
410.bwaves-1963B.csv
123
410.bwaves-2097B.csv
123
                         id         0         1         2        3        4  \
0       400.perlbench-41B_0  48486896  16534944  11269872  7347024  3587856   
1     400.perlbench-41B_418  15126048   5219040   3717200  2277376  1575632   
2     400.perlbench-41B_535  15353376   5205904   3685936  2272784  1578496   
3       400.perlbench-50B_0  55123232  16123168  13797616  8962624  6739840   
4     400.perlbench-50B_493  24602736   7493280   6130544  4009984  3216688   
...                     ...       ...       ...       ...      ...      ...   
7554       657.xz_s-4994B_0   9235744   9865600   4683024  3626736  4171776   
7555     657.xz_s-4994B_174   9843216  10445696   5134928  3935216  4373984   
7556     657.xz_s-4994B_362  10674864 

                         id         0         1         2        3        4  \
0       400.perlbench-41B_0  48486896  16534944  11269872  7347024  3587856   
1     400.perlbench-41B_418  15126048   5219040   3717200  2277376  1575632   
2     400.perlbench-41B_535  15353376   5205904   3685936  2272784  1578496   
3       400.perlbench-50B_0  55123232  16123168  13797616  8962624  6739840   
4     400.perlbench-50B_493  24602736   7493280   6130544  4009984  3216688   
...                     ...       ...       ...       ...      ...      ...   
7554       657.xz_s-4994B_0   9235744   9865600   4683024  3626736  4171776   
7555     657.xz_s-4994B_174   9843216  10445696   5134928  3935216  4373984   
7556     657.xz_s-4994B_362  10674864  11244112   5502048  4232272  4618096   
7557     657.xz_s-4994B_563   8198672   8811632   4190272  3216352  3894096   
7558         657.xz_s-56B_0  26404208  36893024  10084752  2060848  2511312   

            5        6        7        8  ...  3758

                         id         0         1         2        3        4  \
0       400.perlbench-41B_0  48486896  16534944  11269872  7347024  3587856   
1     400.perlbench-41B_418  15126048   5219040   3717200  2277376  1575632   
2     400.perlbench-41B_535  15353376   5205904   3685936  2272784  1578496   
3       400.perlbench-50B_0  55123232  16123168  13797616  8962624  6739840   
4     400.perlbench-50B_493  24602736   7493280   6130544  4009984  3216688   
...                     ...       ...       ...       ...      ...      ...   
7554       657.xz_s-4994B_0   9235744   9865600   4683024  3626736  4171776   
7555     657.xz_s-4994B_174   9843216  10445696   5134928  3935216  4373984   
7556     657.xz_s-4994B_362  10674864  11244112   5502048  4232272  4618096   
7557     657.xz_s-4994B_563   8198672   8811632   4190272  3216352  3894096   
7558         657.xz_s-56B_0  26404208  36893024  10084752  2060848  2511312   

            5        6        7        8  ...  3758

                         id         0         1         2        3        4  \
0       400.perlbench-41B_0  48486896  16534944  11269872  7347024  3587856   
1     400.perlbench-41B_418  15126048   5219040   3717200  2277376  1575632   
2     400.perlbench-41B_535  15353376   5205904   3685936  2272784  1578496   
3       400.perlbench-50B_0  55123232  16123168  13797616  8962624  6739840   
4     400.perlbench-50B_493  24602736   7493280   6130544  4009984  3216688   
...                     ...       ...       ...       ...      ...      ...   
7554       657.xz_s-4994B_0   9235744   9865600   4683024  3626736  4171776   
7555     657.xz_s-4994B_174   9843216  10445696   5134928  3935216  4373984   
7556     657.xz_s-4994B_362  10674864  11244112   5502048  4232272  4618096   
7557     657.xz_s-4994B_563   8198672   8811632   4190272  3216352  3894096   
7558         657.xz_s-56B_0  26404208  36893024  10084752  2060848  2511312   

            5        6        7        8  ...  3758

470.lbm-1274B.csv
123
471.omnetpp-188B.csv
123
473.astar-153B.csv
123
473.astar-359B.csv
123
473.astar-42B.csv
123
481.wrf-1170B.csv
123
481.wrf-1254B.csv
123
481.wrf-1281B.csv
123
481.wrf-196B.csv
123
481.wrf-455B.csv
123
481.wrf-816B.csv
123
482.sphinx3-1100B.csv
123
482.sphinx3-1297B.csv
123
482.sphinx3-1395B.csv
123
482.sphinx3-1522B.csv
123
482.sphinx3-234B.csv
123
482.sphinx3-417B.csv
123
483.xalancbmk-127B.csv
123
483.xalancbmk-716B.csv
123
483.xalancbmk-736B.csv
123
600.perlbench_s-1273B.csv
123
600.perlbench_s-210B.csv
123
600.perlbench_s-570B.csv
123
602.gcc_s-1850B.csv
123
602.gcc_s-2226B.csv
123
602.gcc_s-2375B.csv
123
602.gcc_s-734B.csv
123
603.bwaves_s-1080B.csv
123
603.bwaves_s-1740B.csv
123
603.bwaves_s-2609B.csv
123
603.bwaves_s-2931B.csv
123
                         id         0         1         2        3        4  \
0       400.perlbench-41B_0  48486896  16534944  11269872  7347024  3587856   
1     400.perlbench-41B_418  15126048   5219040   3717200  2277376  1575

                         id         0         1         2        3        4  \
0       400.perlbench-41B_0  48486896  16534944  11269872  7347024  3587856   
1     400.perlbench-41B_418  15126048   5219040   3717200  2277376  1575632   
2     400.perlbench-41B_535  15353376   5205904   3685936  2272784  1578496   
3       400.perlbench-50B_0  55123232  16123168  13797616  8962624  6739840   
4     400.perlbench-50B_493  24602736   7493280   6130544  4009984  3216688   
...                     ...       ...       ...       ...      ...      ...   
7554       657.xz_s-4994B_0   9235744   9865600   4683024  3626736  4171776   
7555     657.xz_s-4994B_174   9843216  10445696   5134928  3935216  4373984   
7556     657.xz_s-4994B_362  10674864  11244112   5502048  4232272  4618096   
7557     657.xz_s-4994B_563   8198672   8811632   4190272  3216352  3894096   
7558         657.xz_s-56B_0  26404208  36893024  10084752  2060848  2511312   

            5        6        7        8  ...  3758

641.leela_s-334B.csv
123
641.leela_s-602B.csv
123
                         id         0         1         2        3        4  \
0       400.perlbench-41B_0  48486896  16534944  11269872  7347024  3587856   
1     400.perlbench-41B_418  15126048   5219040   3717200  2277376  1575632   
2     400.perlbench-41B_535  15353376   5205904   3685936  2272784  1578496   
3       400.perlbench-50B_0  55123232  16123168  13797616  8962624  6739840   
4     400.perlbench-50B_493  24602736   7493280   6130544  4009984  3216688   
...                     ...       ...       ...       ...      ...      ...   
7554       657.xz_s-4994B_0   9235744   9865600   4683024  3626736  4171776   
7555     657.xz_s-4994B_174   9843216  10445696   5134928  3935216  4373984   
7556     657.xz_s-4994B_362  10674864  11244112   5502048  4232272  4618096   
7557     657.xz_s-4994B_563   8198672   8811632   4190272  3216352  3894096   
7558         657.xz_s-56B_0  26404208  36893024  10084752  2060848  2511312   

 

644.nab_s-7928B.csv
123
644.nab_s-9322B.csv
123
                         id         0         1         2        3        4  \
0       400.perlbench-41B_0  48486896  16534944  11269872  7347024  3587856   
1     400.perlbench-41B_418  15126048   5219040   3717200  2277376  1575632   
2     400.perlbench-41B_535  15353376   5205904   3685936  2272784  1578496   
3       400.perlbench-50B_0  55123232  16123168  13797616  8962624  6739840   
4     400.perlbench-50B_493  24602736   7493280   6130544  4009984  3216688   
...                     ...       ...       ...       ...      ...      ...   
7554       657.xz_s-4994B_0   9235744   9865600   4683024  3626736  4171776   
7555     657.xz_s-4994B_174   9843216  10445696   5134928  3935216  4373984   
7556     657.xz_s-4994B_362  10674864  11244112   5502048  4232272  4618096   
7557     657.xz_s-4994B_563   8198672   8811632   4190272  3216352  3894096   
7558         657.xz_s-56B_0  26404208  36893024  10084752  2060848  2511312   

   

KeyboardInterrupt: 

In [5]:
print(out_df)
out_df.to_csv('output_512/dataset.csv',index=False)

                         id         0         1         2        3        4  \
0       400.perlbench-41B_0  48486896  16534944  11269872  7347024  3587856   
1     400.perlbench-41B_418  15126048   5219040   3717200  2277376  1575632   
2     400.perlbench-41B_535  15353376   5205904   3685936  2272784  1578496   
3       400.perlbench-50B_0  55123232  16123168  13797616  8962624  6739840   
4     400.perlbench-50B_493  24602736   7493280   6130544  4009984  3216688   
...                     ...       ...       ...       ...      ...      ...   
7554       657.xz_s-4994B_0   9235744   9865600   4683024  3626736  4171776   
7555     657.xz_s-4994B_174   9843216  10445696   5134928  3935216  4373984   
7556     657.xz_s-4994B_362  10674864  11244112   5502048  4232272  4618096   
7557     657.xz_s-4994B_563   8198672   8811632   4190272  3216352  3894096   
7558         657.xz_s-56B_0  26404208  36893024  10084752  2060848  2511312   

            5        6        7        8  ...  3758

In [4]:
dct = {}
for i in out_df["id"]:
    if i in dct:
        print("Duplicate", i)
    dct[i] = 1